In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
def get_mean(table):
    # функция для нахождения средних значений
    mask = table["Condition"] == "Ctrl"
    return np.mean(table[mask]).values[0], np.mean(table[~mask]).values[0]

def FoldChange(table, GOI, Reference, E):
    '''table: таблица со значениями Ct для различных условий различных генов;
    GOI: gene of interest (str, как записано в table)
    Reference: референсный ген (str, как записано в table)
    E: словарь {ген: эффективность}'''
    
    table["Ct"] = table["Ct"].astype(float)
    Target = table[table["Gene"]==GOI].drop("Gene", axis = 1)
    Ref = table[table["Gene"]==Reference].drop("Gene", axis = 1)
    
    # Считаем ср.арифметическое
    df = pd.DataFrame([get_mean(Target), get_mean(Ref)],
            index = [GOI, Reference],
            columns = ["Ctrl", "Hypo"])
    
    # Вычитаем из контрольного образца рез-т
    df["Ctrl-Hypo"] = df["Ctrl"] - df["Hypo"]
    
    # Находим E^{Ct}
    df["E^Ct"] = np.vstack((
        np.power(E[GOI], df.loc[GOI, "Ctrl-Hypo"]),
        np.power(E[Reference], df.loc[Reference, "Ctrl-Hypo"])
    ))
    
    FC = df.loc["TXNRD1", "E^Ct"] / df.loc["LBR", "E^Ct"]
    print("Fold Change: ", round(FC, 6))
    print("log2 Fold Change: ", round(np.log2(FC), 6))
    
    return df

In [3]:
# Создаем словарь эффективностей для таргетного (TXNRD1) и референсного (LBR) генов

Efficiency = {"TXNRD1": 2, "LBR": 2.11}

#### Our results:

In [4]:
data1 = pd.read_csv("Hypoxia_LBR_TXNRD1_2.txt", sep = '\t')
FoldChange(data1, "TXNRD1", "LBR", Efficiency)

Fold Change:  2.682004
log2 Fold Change:  1.423312


,Ctrl,Hypo,Ctrl-Hypo,E^Ct
TXNRD1,24.433333,23.800000,0.633333,1.551145
LBR,26.700000,27.433333,-0.733333,0.578353


#### Colleague results - 1:

In [5]:
data2 = pd.read_csv("Hypoxia_LBR_TXNRD1_1.txt", sep = '\t')
FoldChange(data2, "TXNRD1", "LBR", Efficiency)

Fold Change:  3.700043
log2 Fold Change:  1.887542


,Ctrl,Hypo,Ctrl-Hypo,E^Ct
TXNRD1,24.033333,23.366667,0.666667,1.587401
LBR,27.100000,28.233333,-1.133333,0.429022


#### Colleague results - 2:

In [6]:
data3 = pd.read_csv("Hypoxia_LBR_TXNRD1_3.txt", sep = '\t')
FoldChange(data3, "TXNRD1", "LBR", Efficiency)

Fold Change:  2.884225
log2 Fold Change:  1.528184


,Ctrl,Hypo,Ctrl-Hypo,E^Ct
TXNRD1,23.800000,22.200000,1.600000,3.031433
LBR,23.233333,23.166667,0.066667,1.051039
